In [3]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt
import corner
import importlib
import logging
import os
from tqdm import tqdm
import keras.backend as K
from functions import load_data, chi2_graph

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
importlib.reload(logging)
gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(gpus[1],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
    print(e)

2 Physical GPUs, 1 Logical GPU


In [4]:
split = 900000
x_train, y_train, x_val, y_val = load_data()

In [5]:
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = tf.keras.Sequential([
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)))])
    return prior_model

def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = tf.keras.Sequential([
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ), tfp.layers.MultivariateNormalTriL(n),])
    return posterior_model

In [6]:
def create_bnn_model(train_size):
    inputs = tf.keras.Input(shape=(len(x_train[0]),), name = 'input')
    features = tf.keras.layers.BatchNormalization()(inputs)
    features = tf.keras.layers.Dense(256, activation='relu')(features)
    features = tf.keras.layers.Dense(128, activation='relu')(features)

    for units in [32]:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)
    features = tf.keras.layers.Dense(16, activation='relu')(features)
    features = tf.keras.layers.Dense(4, activation='linear')(features)
    outputs = features
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

alpha = np.array([1, 20, 1, 1])
alpha = alpha/sum(alpha)

def custom_mse(y_true, y_pred):
    loss = K.square(y_pred - y_true)  
    loss = loss * alpha      
    loss = K.sum(loss, axis=1)        
    return loss

In [8]:
bnn = create_bnn_model(len(x_train))
bnn.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanSquaredError()],
)
bnn.load_weights('./bnn/weight_3.h5')

Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


In [9]:
data = np.load('../../Data/sample_NuFit0911.npz')
data_all = np.column_stack([data['ve_dune'][:,:36], data['vu_dune'][:,:36], data['vebar_dune'][:,:36], data['vubar_dune'][:,:36]])

In [20]:
sample_amount = 100000
prediction_IO = np.array([bnn.predict(data_all[:1, :])[0] for _ in range(sample_amount)])
prediction_NO = np.array([bnn.predict(data_all[1:2, :])[0] for _ in range(sample_amount)])

In [ ]:
# np.savez('./contour_IO/bnn.npz', prediction_IO)
# np.savez('./contour_NO/bnn.npz', prediction_NO)

In [ ]:
x, y, X, Y, Z = chi2_graph()

In [ ]:
range = [[46,52], [175, 225]]
angle = np.angle(prediction_IO[:, 2] + 1j*prediction_IO[:, 3], deg=True)
angle = np.where(angle > 0 , angle, angle+360)

fig, ax = plt.subplots(1,1, figsize=(15,15))
plot_axis = plt.subplot(2,2,1)
plot_axis.hist(prediction_IO[:, 1]*180/np.pi,density=1,bins=x)

plot_axis = plt.subplot(2,2,4)

plot_axis.hist(angle,orientation="horizontal",density=1, bins=y)

plot_axis = plt.subplot(2,2,3)
plot_axis.set_title('One sigma contour')
DUT2 = plot_axis.contour(X, Y, Z, 0, colors='blue', linestyles="-", linewidths=1)
corner.hist2d(prediction_IO[:, 1]*180/np.pi, angle,
                    levels=(0.68,),
                    scale_hist=True,
                    plot_datapoints=False,
                    color='green',
                    labels= ["$\\theta_{23} $($^\circ$)", "$\delta_{cp} $($^\circ$)"],
                    range=range,
                    plot_contours = True,
                    plot_density = False,
                    fontsize=30,
                    bins = [200, 200],
                    label_kwargs={"fontsize": 30},
                    smooth=True
                   )
plt.scatter(data['theta23'][0], data['delta'][0])
plot_axis.set_xlabel(r'$\theta_{23} $', fontsize=20)
plot_axis.set_ylabel(r'$\delta_{cp} $', fontsize=20)
proxy = [plt.Rectangle((2, 2), 0.0, 3, 90, fc=c) for c in ['blue', 'green']]
plt.legend(proxy, ["chi square", "ML"])
plt.show()